In [1]:

import pandas as pd
import numpy as np
import random 
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

CSV_PATH="cleaned_songs.csv"
df=pd.read_csv(CSV_PATH)
random_seed=47

2025-10-18 14:43:48.561721: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-18 14:43:49.048873: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ivan/Desktop/Diplomska/.venv/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/ivan/Deskto

Parameters to test
Best F1 Score: 0.6983445428792973
Best hyperparameters: {'max_features': 2000, 'n_layers': 1, 'neurons': 228, 'activation': 'tanh', 'dropout_rate': 0.11084515268749309, 'optimizer': 'adam', 'lr': 0.00503887829334363, 'l2_reg': 0.00010248432412205469, 'n_epochs': 14}


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix

# -----------------------------
# GPU config
# -----------------------------
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# -----------------------------
# Best hyperparameters from Optuna
# -----------------------------
best_params = {
    'max_features': 2000,
    'n_layers': 1,
    'neurons': 228,
    'activation': 'tanh',
    'dropout_rate': 0.11084515268749309,
    'optimizer': 'adam',
    'lr': 0.00503887829334363,
    'l2_reg': 0.00010248432412205469,
    'n_epochs': 14
}

# -----------------------------
# Load training dataset
# -----------------------------
TRAIN_CSV = "cleaned_songs.csv"  # your original training dataset
df_train = pd.read_csv(TRAIN_CSV)
df_train['song_text'] = df_train['song_text'].str.lower()

# TF-IDF vectorizer fitted on training data
vectorizer = TfidfVectorizer(
    max_features=best_params['max_features'], ngram_range=(1,2), min_df=2, max_df=0.8
)
X_train = vectorizer.fit_transform(df_train['song_text']).toarray()

# Label encoder fitted on training data
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(df_train['author'])

num_classes = len(label_encoder.classes_)

# Reshape for LSTM
X_train = np.expand_dims(X_train, axis=1)

# -----------------------------
# Build model
# -----------------------------
model = keras.Sequential()
model.add(layers.Input(shape=(1, X_train.shape[2])))

# Optional dimensionality reduction
if X_train.shape[2] > 1024:
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Reshape((1, 1024)))

for i in range(best_params['n_layers']):
    return_sequences = i < best_params['n_layers'] - 1
    model.add(layers.LSTM(
        best_params['neurons'],
        activation=best_params['activation'],
        kernel_regularizer=regularizers.l2(best_params['l2_reg']),
        return_sequences=return_sequences
    ))
    model.add(layers.Dropout(best_params['dropout_rate']))

model.add(layers.Dense(num_classes, activation="softmax"))

optimizer = keras.optimizers.Adam(learning_rate=best_params['lr'])
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# -----------------------------
# Train model
# -----------------------------
model.fit(
    X_train, y_train,
    epochs=best_params['n_epochs'],
    batch_size=32,
    verbose=1
)

# -----------------------------
# Load new dataset for evaluation
# -----------------------------
EVAL_CSV = "author_songs_created_only_with_styles.csv"  # new dataset
df_eval = pd.read_csv(EVAL_CSV)
df_eval['new_song'] = df_eval['new_song'].str.lower()

# Transform using training vectorizer
X_eval = vectorizer.transform(df_eval['new_song']).toarray()
X_eval = np.expand_dims(X_eval, axis=1)

# Only evaluate authors present in training
mask = df_eval['author'].isin(label_encoder.classes_)
X_eval = X_eval[mask.values]
y_eval = df_eval['author'][mask].values
y_eval_encoded = label_encoder.transform(y_eval)

# -----------------------------
# Evaluate
# -----------------------------
y_pred = np.argmax(model.predict(X_eval, verbose=0), axis=1)
f1 = f1_score(y_eval_encoded, y_pred, average="weighted")
print("F1 score on evaluation dataset:", f1)

# Optional: confusion matrix
cm = confusion_matrix(y_eval_encoded, y_pred)
print("Confusion matrix:\n", cm)


Training samples: 807, Test samples: 347
Author distribution:
author
Блаже Конески          182
Пијан Славеј           160
Ефтим Клетников        135
Славко Јаневски         70
Владо Урошевиќ          61
Катица Ќулавкова        60
Братислав Ташковски     55
Веле Смилевски          51
Ристо Лазаров           48
Адам Гајтани            47
Бранко Цветкоски        41
Петре М. Андреевски     41
Милош Линдро            40
Тодор Чаловски          31
Ацо Шопов               31
Анте Поповски           21
Гане Тодоровски         19
Богомил Ѓузел           19
Игор Исаковски          15
Матеја Матевски         14
Кочо Рацин              13
Name: count, dtype: int64
